In [7]:
"""
Load Data as 4 images in a 2x2 grid
"""
from torchvision import transforms as T
from random import sample, shuffle
import os
import torch
from minmax.lmm_models import vgg, resnet, densenet
from torchvision.models import vgg as vggn
import minmax.mm_linearize as mm 
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

data_transforms = {
    'train': T.Compose([
        T.Resize((224,224)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'inverse': T.Compose([
        T.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
        T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    ]),
}

# Create ImageNet dataset
from torchvision.datasets import ImageNet
imagenet = ImageNet(
    root='~/datasets/ImageNet',
    split="val",
    transform=data_transforms['train']
)

# Create oxford pets binary classification dataset (cats vs dogs)
from torchvision.datasets import OxfordIIITPet
oxfordpets = OxfordIIITPet(
    root='~/datasets/',
    split='trainval',
    transform=data_transforms['train'],
    target_transform=lambda x: x+1 in [1, 6, 7, 8, 10, 12, 21, 24, 27, 28, 33, 34]
)

class QuartileDataset(torch.utils.data.Dataset):
    """Compose 4 images into one image"""

    def __init__(self, dataset_1, dataset_2):
        """
        Args:
            dataset_1 (Dataset): The main dataset with target images
            dataset_2 (Dataset): The data used to fill the other 3 spots
        """
        self.dataset_1 = dataset_1
        self.dataset_2 = dataset_2
        
    def __len__(self):
        return len(self.dataset_1)
    
    def __getitem__(self, idx):
        """
        ##########
        # 
        # 
        """
        X1, Y = self.dataset_1[idx]
        X2, _ = self.dataset_2[(idx) % len(self.dataset_2)]
        X3, _ = self.dataset_2[(2*idx) % len(self.dataset_2)]
        X4, _ = self.dataset_2[(3*idx) % len(self.dataset_2)]
        corner = idx % 4
        if corner == 0:
            pass
        elif corner == 1:
            X1, X2 = X2, X1
        elif corner == 2:
            X1, X3 = X3, X1
        elif corner == 3:
            X1, X4 = X4, X1
        h1 = torch.cat((X1, X2), dim=2)
        h2 = torch.cat((X3, X4), dim=2)
        X = torch.cat([h1, h2], dim=1)
        return X, (Y, idx % 4)

testloader = torch.utils.data.DataLoader(
    QuartileDataset(oxfordpets, imagenet),
    batch_size=1,
    num_workers=2,
    shuffle=True
)

TRAINED = True
"""
Load a pytorch network (vgg19)
"""
if TRAINED:
    netn = torch.load("finetuned.pt")
    net = vgg.vgg19()
    net.classifier[6] = mm.Linear(4096, 1)
    net.load_state_dict(netn.state_dict())
    print("Loaded trained models")
else:
    netn = vggn.vgg19()
    netn.classifier[6] = nn.Linear(4096, 1)
    net = vgg.vgg19()
    net.classifier[6] = mm.Linear(4096, 1)
    print("Loaded untrained models")

Loaded trained models


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from minmax.mm_linearize import mmTensor, explainV1, explainV2, explainGradient, prediction_score
from sklearn import metrics
from pytorch_grad_cam import GradCAM
import numpy as np
from datetime import datetime
from captum.attr import (
    Occlusion,
    NoiseTunnel,
    DeepLiftShap,
    Saliency,
    GuidedGradCam
)

def auc(y, pred):
    y = y.flatten()
    pred = pred.flatten()
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    return metrics.auc(fpr, tpr)

OUTPUT_FOLDER = "Experiment_3"
try:
    os.mkdir(OUTPUT_FOLDER)
    print("Output Folder Created")
except FileExistsError:
    print("Output Folder Exists")
    
for i, (data, (target, corner)) in enumerate(testloader):
    # Create correct quartile maks
    M1 = torch.ones((224,224))
    M2 = torch.zeros((224,224))
    M3 = torch.zeros((224,224))
    M4 = torch.zeros((224,224))
    if corner == 0:
        pass
    elif corner == 1:
        M1, M2 = M2, M1
    elif corner == 2:
        M1, M3 = M3, M1
    elif corner == 3:
        M1, M4 = M4, M1
    h1 = torch.cat((M1, M2), dim=1)
    h2 = torch.cat((M3, M4), dim=1)
    mask = torch.cat([h1, h2], dim=0)

    # Plot all versions of the explanation
    fig, axs = plt.subplots(2, 4)
    fig.set_size_inches(17,10)
    
    score = torch.sigmoid(prediction_score(net, data))

    # Guided GradCam
    timeGuidedGradCam = datetime.now()
    expl = GuidedGradCam(netn, netn.features[-1])
    explGuidedGradCam = expl.attribute(data)
    explGuidedGradCam = explGuidedGradCam.detach().numpy()
    explGuidedGradCam = explGuidedGradCam.sum(axis=0)
    explGuidedGradCam = np.linalg.norm(explGuidedGradCam, axis=0)
    scoreGuidedGradCam = auc(mask, explGuidedGradCam)
    
    axs[0][0].axis("off")
    axs[0][0].imshow(explGuidedGradCam, cmap=plt.cm.Reds)
    axs[0][0].set_title("Guided-GradCam {:.3f}".format(scoreGuidedGradCam))
    timeGuidedGradCam = (datetime.now() - timeGuidedGradCam).seconds
    if i < 5: plt.imsave("{}/{}_guidedgradcam.png".format(OUTPUT_FOLDER, i), explGuidedGradCam, cmap=plt.cm.Reds)

    # SmoothGrad
    timeSmoothGrad = datetime.now()
    saliency = Saliency(netn)
    nt = NoiseTunnel(saliency)
    explSmoothGrad = nt.attribute(data, nt_type='smoothgrad', nt_samples=15)
    explSmoothGrad = explSmoothGrad.detach().numpy().sum(axis=0)
    explSmoothGrad = np.linalg.norm(explSmoothGrad, axis=0)
    scoreSmoothGrad = auc(mask, explSmoothGrad)
    
    axs[0][1].axis("off")
    axs[0][1].imshow(explSmoothGrad, cmap=plt.cm.Reds)
    axs[0][1].set_title("SmoothGrad {:.3f}".format(scoreSmoothGrad))
    timeSmoothGrad = (datetime.now() - timeSmoothGrad).seconds
    if i < 5: plt.imsave("{}/{}_smoothgrad.png".format(OUTPUT_FOLDER, i), explSmoothGrad, cmap=plt.cm.Reds)
    
    # DeepLiftShap
    timeShapley = datetime.now()
    dist = torch.cat([data * 0, data * 1])
    expl = DeepLiftShap(netn)
    explShapley = expl.attribute(data, baselines=dist)
    explShapley = explShapley.detach().numpy().sum(axis=0)
    explShapley = np.linalg.norm(explShapley, axis=0)
    scoreShapley = auc(mask, explShapley)
    
    axs[0][2].axis("off")
    axs[0][2].imshow(explShapley, cmap=plt.cm.Reds)
    axs[0][2].set_title("Shapley {:.3f}".format(scoreShapley))
    timeShapley = (datetime.now() - timeShapley).seconds
    if i < 5: plt.imsave("{}/{}_shapley.png".format(OUTPUT_FOLDER, i), explShapley, cmap=plt.cm.Reds)

    # Occlusion
    timeOcclusion = datetime.now()
    expl = Occlusion(netn)
    explOcclusion = expl.attribute(data, strides=(3,25,25), sliding_window_shapes=(3,50,50))
    explOcclusion = explOcclusion.detach().numpy().sum(axis=0).sum(axis=0)
    if not target: explOcclusion = -explOcclusion
    explOcclusion -= explOcclusion.min()
    explOcclusion /= explOcclusion.max()
    scoreOcclusion = auc(mask, explOcclusion)
    
    axs[0][3].axis("off")
    axs[0][3].imshow(explOcclusion, cmap=plt.cm.Reds)
    axs[0][3].set_title("Occlusion {:.3f}".format(scoreOcclusion))
    timeOcclusion = (datetime.now() - timeOcclusion).seconds
    if i < 5: plt.imsave("{}/{}_occlusion.png".format(OUTPUT_FOLDER, i), explOcclusion, cmap=plt.cm.Reds)

    # Gradient
    timeGradient = datetime.now()
    explGradient, _ = explainGradient(net, data)
    scoreGradient = auc(mask, explGradient)
    
    axs[1][0].axis("off")
    axs[1][0].imshow(explGradient, cmap=plt.cm.Reds)
    axs[1][0].set_title("Gradient {:.3f}".format(scoreGradient))
    timeGradient = (datetime.now() - timeGradient).seconds
    if i < 5: plt.imsave("{}/{}_gradient.png".format(OUTPUT_FOLDER, i), explGradient, cmap=plt.cm.Reds)

    # RangeGrad
    timeRangeGrad = datetime.now()
    explRangeGrad, _ = explainV2(net, data, scale=1/100, width=1/1000)
    scoreRangeGrad = auc(mask, explRangeGrad)
    axs[1][1].axis("off")
    axs[1][1].imshow(explRangeGrad, cmap=plt.cm.Reds)
    axs[1][1].set_title("RangeGrad {:.3f}".format(scoreRangeGrad))
    timeRangeGrad = (datetime.now() - timeRangeGrad).seconds
    if i < 5: plt.imsave("{}/{}_rangegrad.png".format(OUTPUT_FOLDER, i), explRangeGrad, cmap=plt.cm.Reds)

    # Mask 
    axs[1][2].axis("off")
    axs[1][2].imshow(mask, cmap=plt.cm.Reds)
    axs[1][2].set_title("C: {}".format(corner.item()))
    if i < 5: plt.imsave("{}/{}_mask.png".format(OUTPUT_FOLDER, i), mask, cmap=plt.cm.Reds)

    # Image
    image = data_transforms['inverse'](data)[0].movedim(0,2).numpy()
    axs[1][3].axis("off")
    axs[1][3].imshow(image)
    axs[1][3].set_title("T: {}, P: {}".format(target.item(), score > 0.5))
    plt.show()
    if i < 5: plt.imsave("{}/{}_original.png".format(OUTPUT_FOLDER, i), image)
    
    # Add to data table
    with open("Experiment_3/scores_finetuned.txt", "a") as f:
        f.write("{},{},{},{},{},{}\n".format(scoreGuidedGradCam, scoreSmoothGrad, scoreShapley, scoreOcclusion, scoreGradient, scoreRangeGrad))
    with open("Experiment_3/times_finetuned.txt", "a") as f:
        f.write("{},{},{},{},{},{}\n".format(timeGuidedGradCam, timeSmoothGrad, timeShapley, timeOcclusion, timeGradient, timeRangeGrad))

Output Folder Exists


In [1]:
import csv
import statistics as stats
import scipy.stats as st
from scipy.stats import t
import numpy as np

# define the file path and read the file
filepath = 'Experiment_3/scores_finetuned.txt'
data = []
with open(filepath, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        data.append(row)

# define the file path and read the file
filepath = 'Experiment_3/times_finetuned.txt'
time_data = []
with open(filepath, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        time_data.append(row)

# transpose the data to make columns into rows
data = list(map(list, zip(*data)))
time_data = list(map(list, zip(*time_data)))
labels = ["Guided-GradCam", "SmoothGrad", "DeepLiftShapley", "Occlusion", "Gradient", "RangeGrad"]

print("\\begin{tabular}{r|rl|rl}")
print("\tMethod & \\multicolumn{2}{c}{AUC} & \\multicolumn{2}{c}{Runtime (s)} \\\\\\hline")
for i in range(6):
    # calculate mean and confidence interval
    d = [float(j) for j in data[i]]
    time_d = [float(j) for j in time_data[i]]
    mean = stats.mean(d)
    ci = t.interval(0.95, len(d)-1, scale=st.sem(d))
    time_mean = stats.mean(time_d)
    time_ci = t.interval(0.95, len(time_d)-1, scale=st.sem(time_d))
    print("\t{} & {:.3f} & \\textpm {:.3f} & {:.1f} & \\textpm {:.1f} \\\\".format(labels[i], mean, ci[1], time_mean, time_ci[1]))
print("\\end{tabular}")

\begin{tabular}{r|rl|rl}
	Method & \multicolumn{2}{c}{AUC} & \multicolumn{2}{c}{Runtime (s)} \\\hline
	Guided-GradCam & 0.571 & \textpm 0.012 & 1.1 & \textpm 0.0 \\
	SmoothGrad & 0.590 & \textpm 0.010 & 17.2 & \textpm 0.0 \\
	DeepLiftShapley & 0.698 & \textpm 0.008 & 8.1 & \textpm 0.0 \\
	Occlusion & 0.610 & \textpm 0.006 & 83.7 & \textpm 0.1 \\
	Gradient & 0.684 & \textpm 0.009 & 6.0 & \textpm 0.0 \\
	RangeGrad & 0.663 & \textpm 0.010 & 10.8 & \textpm 0.0 \\
\end{tabular}
